In [5]:
import pandas as pd
import ccxt
from backtesting import Strategy

/usr/local/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
def data_to_df(data):
    """CCXTから取得したデータをDataFrameに変換"""

    df = pd.DataFrame(
        data, columns=["Date Time", "Open", "High", "Low", "Close", "Volume"]
    )
    df["Date Time"] = pd.to_datetime(df["Date Time"] / 1000, unit="s")
    df.set_index("Date Time", inplace=True)
    return df


def get_data(ticker, timeframe="5m"):
    # 取引所のリミットレート(アクセス制限)を超えないように設定
    exchange = ccxt.ftx({"enableRateLimit": True})
    df = data_to_df(
        exchange.fetch_ohlcv(
            ticker, 
            timeframe=timeframe, 
            params={"reverse": False})
    )
    return df

In [4]:
# データが取得できることを確認 
get_data("BNB-PERP")

,Open,High,Low,Close,Volume
Date Time,,,,,
2021-12-25 02:25:00,544.8600,545.6475,544.8600,545.4500,67045.67325
2021-12-25 02:30:00,545.4500,546.2500,545.4450,545.9100,73218.93850
2021-12-25 02:35:00,545.9100,546.0175,545.0425,545.1875,172273.27275
2021-12-25 02:40:00,545.1875,545.3325,544.6125,544.6350,52970.33600
2021-12-25 02:45:00,544.6350,544.6975,543.3425,543.3450,59691.80375
...,...,...,...,...,...
2021-12-30 07:05:00,519.4275,519.7275,518.4400,518.7475,629922.45725
2021-12-30 07:10:00,518.7475,519.3650,518.4950,519.3200,359471.32300
2021-12-30 07:15:00,519.3150,519.3150,518.0000,518.0300,408406.70425


In [6]:
def SMA(values, n):
    # I メソッド経由で渡ってくるデータはただの array なので、
    # pandas.Series メソッドへ変換して移動平均を作る
    return pd.Series(values).rolling(n).mean()


class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        # まずは SMA を2本計算
        self.sma1 = self.I(SMA, self.data["Close"], self.n1)  # 速い移動平均
        self.sma2 = self.I(SMA, self.data["Close"], self.n2)  # 遅い移動平均

    def next(self):
        # next メソッドは、バー毎に実行される
        # 最新のバーは[-1]でアクセスできる。

        # 速い移動平均が遅い移動平均をうわ抜いたタイミングであれば、
        if self.sma1[-2] < self.sma2[-2] and self.sma1[-1] > self.sma2[-1]:
            # その時持っていたポジションをクローズして
            self.position.close()
            # ロングポジションを取る
            self.buy()

        # その逆
        elif self.sma1[-2] > self.sma2[-2] and self.sma1[-1] < self.sma2[-1]:
            self.position.close()
            self.sell()

In [7]:
from backtesting import Backtest

df = get_data("BNB-PERP")
bt = Backtest(
    df,  # データ
    SmaCross,  # ストラテジークラス
    cash=10000,  # 初期投資額
    commission=0.002,  # 取引手数料
    trade_on_close = True,  # False にするとシグナル発生後の次のバーのオープンで取引をする。default は False.
)
# バックテスト実行
stats = bt.run() 
# 結果を出力
print(stats)

Start                     2021-12-25 02:25:00
End                       2021-12-30 07:25:00
Duration                      5 days 05:00:00
Exposure Time [%]                    98.53431
Equity Final [$]                   9045.93464
Equity Peak [$]                   10019.07577
Return [%]                          -9.540654
Buy & Hold Return [%]               -4.747915
Return (Ann.) [%]                  -95.537266
Volatility (Ann.) [%]                3.023249
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -9.997515
Avg. Drawdown [%]                   -5.161904
Max. Drawdown Duration        5 days 02:00:00
Avg. Drawdown Duration        2 days 13:33:00
# Trades                                   77
Win Rate [%]                        20.779221
Best Trade [%]                       3.645212
Worst Trade [%]                     -1.838527
Avg. Trade [%]                    

In [8]:
# 資金の増減
stats["_equity_curve"].tail()

,Equity,DrawdownPct,DrawdownDuration
Date Time,,,
2021-12-30 07:05:00,9049.46214,0.096777,NaT
2021-12-30 07:10:00,9059.19464,0.095805,NaT
2021-12-30 07:15:00,9037.26464,0.097994,NaT
2021-12-30 07:20:00,9045.93464,0.097129,NaT
2021-12-30 07:25:00,9045.93464,0.097129,5 days 02:00:00


In [9]:
# 取引履歴
stats["_trades"].head()

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,18,21,48,544.955235,544.8525,-1.84923,-0.000189,2021-12-25 04:10:00,2021-12-25 06:25:00,0 days 02:15:00
1,-18,48,55,543.762795,545.7400,-35.58969,-0.003636,2021-12-25 06:25:00,2021-12-25 07:00:00,0 days 00:35:00
2,18,55,75,546.831480,546.0275,-14.47164,-0.001470,2021-12-25 07:00:00,2021-12-25 08:40:00,0 days 01:40:00
3,-18,75,81,544.935445,546.6225,-30.36699,-0.003096,2021-12-25 08:40:00,2021-12-25 09:10:00,0 days 00:30:00
4,18,81,92,547.715745,545.7675,-35.06841,-0.003557,2021-12-25 09:10:00,2021-12-25 10:05:00,0 days 00:55:00


In [10]:
# 結果は _results.values でアクセス可
bt._results.values

array([Timestamp('2021-12-25 02:25:00'), Timestamp('2021-12-30 07:25:00'),
       Timedelta('5 days 05:00:00'), 98.53431045969354, 9045.934639999989,
       10019.075770000001, -9.540653600000114, -4.747914565954722,
       -95.53726621514672, 3.023248840898634, 0.0, 0.0, 0.0,
       -9.997515269814283, -5.161903784907124,
       Timedelta('5 days 02:00:00'), Timedelta('2 days 13:33:00'), 77,
       20.77922077922078, 3.645211717538266, -1.838526671923657,
       -0.13572353294395434, Timedelta('0 days 10:45:00'),
       Timedelta('0 days 01:36:00'), 0.6019830848778311,
       -0.1323742329593021, -1.4446424882896718, <Strategy SmaCross>,
                                 Equity  DrawdownPct DrawdownDuration
       Date Time
       2021-12-25 02:25:00  10000.00000     0.000000              NaT
       2021-12-25 02:30:00  10000.00000     0.000000              NaT
       2021-12-25 02:35:00  10000.00000     0.000000              NaT
       2021-12-25 02:40:00  10000.00000     0.000000    

In [11]:
# 結果を描画
# SmaCross の init メソッド内で定義した、sma 2つ描画されることを確認
bt.plot()

Row(id='1450', ...)